## LinkedIn Profile Visit and Connection Request

In [10]:
import os, time
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [11]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [12]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [13]:

url = "https://www.linkedin.com/company/kpmg-it-service-gmbh/people/"
driver.get(url)

In [14]:
# Functions 
VISITED_PROFILES_FILE = "visited_profiles.txt"

def load_visited_profiles():
    """Load previously visited profiles from file"""
    if os.path.exists(VISITED_PROFILES_FILE):
        with open(VISITED_PROFILES_FILE, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    return set()

def save_visited_profile(url):
    """Save a visited profile URL to file"""
    with open(VISITED_PROFILES_FILE, 'a', encoding='utf-8') as f:
        f.write(url + '\n')

def visit_and_connect(driver, profile_url,mode,timeout):
    """
    Visit a LinkedIn profile and attempt to connect or follow.
    
    Args:
        driver: Selenium WebDriver instance
        profile_url: LinkedIn profile URL
        timeout: Wait timeout in seconds (default: 5)
    
    Returns:
        str: Action taken ('connect', 'follow', 'already_pending', 
             'already_following', 'already_connected', 'none_found')
    """
    driver.get(profile_url)
    wait = WebDriverWait(driver, timeout)
    time.sleep(2)
    action_taken = None

    # Check if already connected or following
    try:
        # Check for "Pending" button (connection request already sent)
        pending_btn = driver.find_element(By.XPATH, "//main//button[.//span[normalize-space()='Pending']]")
        print("⏳ Connection request already sent (Pending)")
        action_taken = 'already_pending'
        # return action_taken
    except NoSuchElementException:
        pass

    # Check for "Message" button without Connect/Follow (already connected)
    if action_taken is None:
        try:
            message_btn = driver.find_element(By.XPATH, "//main//button[.//span[normalize-space()='Message']]")
            # Check if Connect or Follow buttons exist
            connect_exists = len(driver.find_elements(By.XPATH, "//main//button[.//span[normalize-space()='Connect']]")) > 0
            follow_exists = len(driver.find_elements(By.XPATH, "//main//button[.//span[normalize-space()='Follow']]")) > 0
            
            if not connect_exists and not follow_exists:
                print("✓ Already connected with this person")
                action_taken = 'already_connected'
                # return action_taken
        except NoSuchElementException:
            pass

    # Only proceed if no action has been taken yet
    if action_taken is None:
        if mode == 'connect':
        # Try Connect button
            try:
                connect_btn = wait.until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//main//button[.//span[normalize-space()='Connect']]")
                    )
                )
                print(f"📤 Connect button found")
                connect_btn.click()
                print("   Connect button clicked")
                
                # Handle the "Add a note" modal
                try:
                    send_without_note_btn = wait.until(
                        EC.element_to_be_clickable(
                            (By.XPATH, "//button[.//span[normalize-space()='Send without a note']]")
                        )
                    )
                    send_without_note_btn.click()
                    print("✓ Sent connection request without a note")
                    time.sleep(2)
                    action_taken = 'connect'
                    
                except TimeoutException:
                    print("⚠️ 'Send without a note' button not found")
                    # Close modal if it exists
                    try:
                        close_btn = driver.find_element(By.XPATH, "//button[@aria-label='Dismiss']")
                        close_btn.click()
                    except:
                        pass
                
            except TimeoutException:
                print("   Connect button not found in main profile")
            
        # Try Follow button
        try:
            follow_btn = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//main//button[.//span[normalize-space()='Follow']]")
                )
            )
            follow_btn.click()
            print("✓ Follow button clicked successfully")
            action_taken = 'follow'
            
        except TimeoutException:
            print("❌ Follow button not found either")
            action_taken = 'visited_only'
    return action_taken

In [15]:
# Load previously visited profiles
seen_urls = load_visited_profiles()
print(f"📂 Loaded {len(seen_urls)} previously visited profiles from file")

# Store the main window handle
main_window = driver.current_window_handle
total_connects = 0
total_follows = 0
visit_only = 0
total_profiles_visit = 0

while True:
    # collect profiles currently loaded
    profile_links = driver.find_elements(
        By.XPATH,
        "//a[contains(@href, 'linkedin.com/in/')]"
    )
    
    # Extract all URLs first and remove duplicates
    profile_urls = []
    for link in profile_links:
        href = link.get_attribute("href")
        if href:
            # Clean URL: remove query parameters
            clean_url = href.split('?')[0].rstrip('/')
            if clean_url not in seen_urls:
                profile_urls.append(clean_url)
    
    total += len(profile_urls)
    print(f"\n✅ Found {len(profile_urls)} new profiles to visit")
    print(f"📊 Total tracked profiles: {total}")
    
    # Now visit each URL in a new tab
    for i, url in enumerate(profile_urls, 1):
        if url in seen_urls:
            print(f"⏭️  [{i}/{len(profile_urls)}] Already visited, skipping: {url}")
            continue
            
        print(f"\n{'='*60}")
        print(f"🔍 [{i}/{len(profile_urls)}] Visiting Profile: {url}")
        print(f"{'='*60}")

        try:
            # Open profile in new tab
            driver.execute_script("window.open('');")
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[-1])
            
            # Now navigate to the profile

            # try clicking "Show more results"
            if total_connects >= 50:
                print("\n🚫 Reached 50 profiles daily limit.")
                status = visit_and_connect(driver, url,mode='follow', timeout=5)  # mode = 'connect' or 'follow'
            else:
                status = visit_and_connect(driver, url,mode='connect', timeout=5) # mode = 'connect' or 'follow'
            

            print(f"🔔 Action taken: {status}")
            # Update counters based on action taken
            if status == 'connect':
                total_connects += 1
                print(f"📈 Total connection requests sent: {total_connects}")
            elif status == 'follow':
                total_follows += 1
                print(f"📈 Total follows done: {total_follows}")
            elif status == 'visited_only':
                total_profiles_visit += 1
                print(f"📈 Total profiles visited only (no follow/connect): {total_profiles_visit}")

            total_profiles_visit += 1
            time.sleep(2)
            
            # Close tab and switch back to main window
            driver.close()
            driver.switch_to.window(main_window)
            time.sleep(1)
            
            # Mark as seen in memory AND save to file
            seen_urls.add(url)
            save_visited_profile(url)
            print(f"✅ Profile saved to tracking file")
            
        except Exception as e:
            print(f"❌ Error visiting profile: {e}")
            # Still mark as seen to avoid retry loops
            seen_urls.add(url)
            save_visited_profile(url)
            # Make sure we're back on main window
            if len(driver.window_handles) > 1:
                driver.close()
            driver.switch_to.window(main_window)
    
    try:
        show_more = driver.find_element(
            By.XPATH,
            "//button[.//span[text()='Show more results']]"
        )
        print("\n📜 Loading more results...")
        driver.execute_script("arguments[0].click();", show_more)
        time.sleep(3)
    except NoSuchElementException:
        print("\n🎯 No more results to load. Finished!")
        break

    if total_profiles_visit >= 500:
        print("\n🚫 Reached 100 profiles visit limit for this session.")
        break

print(f"\n{'='*60}")
print(f"✅ COMPLETED!")
print(f"📊 Total Connects: {total_connects}")
print(f"📊 Total Follows: {total_follows}")
print(f"📊 Total Visit Only: {visit_only}")
print(f"📊 Total profiles visited: {total_profiles_visit}")
print(f"{'='*60}")

📂 Loaded 498 previously visited profiles from file

✅ Found 0 new profiles to visit
📊 Total tracked profiles: 91

📜 Loading more results...

✅ Found 0 new profiles to visit
📊 Total tracked profiles: 91

📜 Loading more results...

✅ Found 14 new profiles to visit
📊 Total tracked profiles: 105

🔍 [1/14] Visiting Profile: https://www.linkedin.com/in/anke-poehls-5b710214
   Connect button not found in main profile
✓ Follow button clicked successfully
🔔 Action taken: follow
📈 Total follows done: 1
✅ Profile saved to tracking file
⏭️  [2/14] Already visited, skipping: https://www.linkedin.com/in/anke-poehls-5b710214

🔍 [3/14] Visiting Profile: https://www.linkedin.com/in/nawras-dahdal-de
📤 Connect button found
   Connect button clicked
✓ Sent connection request without a note
❌ Follow button not found either
🔔 Action taken: visited_only
📈 Total profiles visited only (no follow/connect): 2
✅ Profile saved to tracking file
⏭️  [4/14] Already visited, skipping: https://www.linkedin.com/in/nawra